In [1]:
!pip install datasets>=1.18.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer


  Using cached transformers-4.11.3-py3-none-any.whl (2.9 MB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-311
      creating build/lib.linux-x86_64-cpython-311/tokenizers
      copying py_src/tokenizers/__init__.py -> build/lib.linux-x86_64-cpython-311/tokenizers
      creating build/lib.linux-x86_64-cpython-311/tokenizers/models
      copying py_src/tokenizers/models/__init__.py -> build/lib.linux-x86_64-cpython-311/tokenizers/models
      creating build/lib.linux-x86_64-cpython-311/tokenizers/decoders
    

In [1]:
from datasets import load_dataset, Dataset, Audio, load_from_disk
import numpy as np
import json
import os
import re
#test_dataset = load_from_disk("/data/speech_datasets/timit_test_dataset_noisy")

test_dataset = load_from_disk("/mount/arbeitsdaten/studenten1/team-lab-phonetics/2023/student_directories/tran/asr_timit/speech_datasets/timit_test_dataset_background_noisy")


/mount/arbeitsdaten/studenten1/team-lab-phonetics/2023/student_directories/tran/teamlab_venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import AutoModelForCTC

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-fine-tuned").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-fine-tuned")


Downloading: 100%|██████████| 1.90k/1.90k [00:00<00:00, 4.84MB/s]
Downloading: 100%|██████████| 360M/360M [00:09<00:00, 39.4MB/s] 
Downloading: 100%|██████████| 290/290 [00:00<00:00, 910kB/s]
Downloading: 100%|██████████| 23.0/23.0 [00:00<00:00, 80.1kB/s]
Downloading: 100%|██████████| 309/309 [00:00<00:00, 859kB/s]


In [3]:
from datasets import load_metric
wer_metric = load_metric("wer")

/tmp/ipykernel_3074492/2912728219.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [4]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [5]:
import torch
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values_noisy_0"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

results = test_dataset.map(map_to_result)

In [6]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))


Test WER: 0.612


In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [11]:
import IPython.display as ipd
import numpy as np

ipd.Audio(data=results[30]["input_values_noisy_-10"], autoplay=True, rate=16000)

| Dataset                  | WER    |
| ------------------------ | ------ |
| Background Noise -20 dB  | 0.255  |
| Background Noise -10 dB  | 0.337  |
| Background Noise   0 dB  | 0.604  |

